In [4]:
from desc.objectives import ParticleTracer, ObjectiveFunction
from desc.examples import get
from desc.grid import Grid
import desc.io
from desc.backend import jnp
import matplotlib.pyplot as plt
import numpy as np

import jax.numpy as jnp

In [5]:
eq = desc.io.load("test_run.h5")
eq._iota = eq.get_profile("iota").to_powerseries(order=eq.L, sym=True)
eq._current = None
# eq.solve()

In [7]:
tmin = 0
tmax = 0.00007
nt = 20
time = jnp.linspace(tmin, tmax, nt)
psi_i = 0.7
theta_i = 0.2
zeta_i = 0.2

mass = 1.673e-27
Energy = 3.52e6*1.6e-19
ini_vpar = 0.5*jnp.sqrt(2*Energy/mass)
ini_cond = [psi_i, theta_i, zeta_i, float(ini_vpar)]

mass_charge = mass/1.6e-19

grid = Grid(jnp.array([jnp.sqrt(psi_i), theta_i, zeta_i]).T, jitable=True, sort=False)
data = eq.compute("|B|", grid=grid)

mu = Energy/(mass*data["|B|"]) - (ini_vpar**2)/(2*data["|B|"])

ini_param = [float(mu), mass_charge]

objective = ParticleTracer(eq=eq, output_time=time, initial_conditions=ini_cond, initial_parameters=ini_param)


In [8]:
objective.build()
solution = objective.compute(*objective.xs(eq))

print("*************** SOLUTION .compute() ***************")
print(solution)
print("***************************************************")

2023-09-21 00:44:58.995968: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit__odeint_wrapper] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2023-09-21 00:47:32.344654: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 4m33.357390913s

********************************
[Compiling module jit__odeint_wrapper] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


*************** SOLUTION .compute() ***************
[[7.00000000e-01 2.00000000e-01 2.00000000e-01 1.29738345e+07]
 [6.24138006e-01 1.25828369e+00 2.92017378e+00 1.21268279e+07]
 [5.19032052e-01 2.37462789e+00 5.60196862e+00 1.07267094e+07]
 [4.97139260e-01 3.53642771e+00 7.97353540e+00 1.04755540e+07]
 [5.82467364e-01 4.66755200e+00 1.03983780e+01 1.16631459e+07]
 [6.82495771e-01 5.77031228e+00 1.30999265e+01 1.28384553e+07]
 [6.82555490e-01 6.81015022e+00 1.58253491e+01 1.28241246e+07]
 [5.83272707e-01 7.89660498e+00 1.85407618e+01 1.16931841e+07]
 [5.00201288e-01 9.02222466e+00 2.11735978e+01 1.04446790e+07]
 [5.19459950e-01 1.01604040e+01 2.34734990e+01 1.07138840e+07]
 [6.21473589e-01 1.13004207e+01 2.60074270e+01 1.21301376e+07]
 [6.97009796e-01 1.24010746e+01 2.87259989e+01 1.29710984e+07]
 [6.58614050e-01 1.34425409e+01 3.14449184e+01 1.25391566e+07]
 [5.46482506e-01 1.45398023e+01 3.41535319e+01 1.12117386e+07]
 [4.89855302e-01 1.56997421e+01 3.67090489e+01 1.03749380e+07]
 [5